## pdf

In [1]:
import PyPDF2

In [ ]:
input = "data/Temu _ Manage orders (1).pdf"
image = "data/Caution (1).png"

with open(input, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text = page.extract_text()
        print(f"Page {page_num + 1}: {text}")
        


Page 1: TEMU-Fulfilment
Destination
David Potts
37 Lawrence Street
Caerphilly
CF83 3AJ
Date 2025-12-03
Weight in kg 2
Reference 1 EVRi836565020135424091 PNBT
COU-PNET
VAN 70
DROP 79
C-ROUND 1391
H-055B-A-001668053-0
Page 2: Purchase Order ID:
PO-210-15775625549432336Order Date:
Dec 3, 2025, 7:58 am GMT
Product DetailsTracking Number:H055BA0016680530 Page 1 / 1 Pages
Shipping Service:EVRI
Product Details Qty Ordered
Personalised Mug - Add Your Text, Picture, Design or Logo on This Ceramic Coffee Mug 11oz.
Customised Gift idea for Birthday, Wedding, Anniversary and for Other Special Moments
Goods SKU ID:56914759170536 Product SKU ID:769188473183
Contribution SKU:-
Attributes:Custom product
(For full customization details, please refer to the order page in the Seller Center.)Text: Anikó's mug DO NOT TOUCH!
Color: {"red":0,"green":0,"blue":0}
Text: Anikó's mug DO NOT TOUCH!
Color: {"red":0,"green":0,"blue":0}1
Page 3: TEMU-Fulfilment
Destination
Glynis Maunder
67 Lea Farm Drive
Leeds
LS5 3

In [9]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io

input_pdf_path = "data/Temu _ Manage orders (1).pdf"
image_path = "data/Caution (1).png"
output_pdf_path = "data/output_with_caution.pdf"

# Charger le PDF original
with open(input_pdf_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    writer = PyPDF2.PdfWriter()

    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]

        # Extraire texte de la page
        text = page.extract_text() or ""

        # Vérifier si la page contient "TEMU-Fulfilment"
        if "TEMU-Fulfilment" in text:
            print(f"✔ Image ajoutée sur page {page_num + 1}")

            page_width = float(page.mediabox[2])
            page_height = float(page.mediabox[3])

            # PDF temporaire pour superposer l’image
            packet = io.BytesIO()
            can = canvas.Canvas(packet, pagesize=(page_width, page_height))

            # Charger l'image
            img = ImageReader(image_path)
            img_width, img_height = img.getSize()

            # Taille réduite (plus petit)
            scale = 0.07   # ← Réduire encore si tu veux (0.05 pour minuscule)
            new_width = img_width * scale
            new_height = img_height * scale

            # Position en haut à droite
            x_pos = page_width - new_width - 15
            y_pos = page_height - new_height - 30

            # Dessiner l'image
            can.drawImage(image_path, x_pos, y_pos, width=new_width, height=new_height)

            can.save()

            # Fusionner avec la page
            packet.seek(0)
            overlay_pdf = PyPDF2.PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)

        else:
            print(f"⏭ Aucune image sur page {page_num + 1}")

        writer.add_page(page)

    # Sauvegarder le PDF final
    with open(output_pdf_path, "wb") as out_file:
        writer.write(out_file)

print("🎉 Fichier terminé : image ajoutée uniquement sur les pages contenant 'TEMU-Fulfilment'.")


✔ Image ajoutée sur page 1
⏭ Aucune image sur page 2
✔ Image ajoutée sur page 3
⏭ Aucune image sur page 4
✔ Image ajoutée sur page 5
⏭ Aucune image sur page 6
✔ Image ajoutée sur page 7
⏭ Aucune image sur page 8
✔ Image ajoutée sur page 9
⏭ Aucune image sur page 10
✔ Image ajoutée sur page 11
⏭ Aucune image sur page 12
✔ Image ajoutée sur page 13
⏭ Aucune image sur page 14
🎉 Fichier terminé : image ajoutée uniquement sur les pages contenant 'TEMU-Fulfilment'.


In [27]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
import re
from PyPDF2 import PageObject

# ------------------ PATHS ------------------
input_pdf_path = "data/Temu _ Manage orders (1).pdf"
image_path = "data/Caution (1).png"
output_pdf_path = "data/output_final.pdf"
guide_pdf_path = "data/4.pdf"
sorted_output_path = "data/output_sorted.pdf"

# ------------------ FUNCTIONS ------------------
def extract_po_id(text):
    """
    Extrait un PO-ID depuis une page contenant "PO-"
    Exemple : "PO-210-15775625549432336"
    """
    match = re.search(r"PO-\d+-\d+", text)
    return match.group(0) if match else None

def extract_id_from_page_text(text):
    """
    Extrait un ID sous forme de chiffres + tirets
    Exemple : 210-15775625549432336
    """
    match = re.search(r"\d+-[\d-]+", text)
    return match.group(0) if match else None

def clone_page(page):
    """
    Crée une copie indépendante d’un PageObject pour éviter les erreurs de flux fermé
    """
    new_page = PageObject.create_blank_page(width=page.mediabox.width, height=page.mediabox.height)
    new_page.merge_page(page)
    return new_page

# ------------------ 1️⃣ Traitement PDF principal (ajout image + PO-ID) ------------------
with open(input_pdf_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    writer = PyPDF2.PdfWriter()
    page_num = 0
    num_pages = len(reader.pages)

    while page_num < num_pages:
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        po_id = None
        skip_next = False

        if "TEMU-Fulfilment" in text:
            # Vérifier la page suivante pour le PO-ID
            if page_num + 1 < num_pages:
                next_page = reader.pages[page_num + 1]
                next_text = next_page.extract_text() or ""
                po_id = extract_po_id(next_text)
                if po_id:
                    skip_next = True
                    print(f"✔ Page {page_num+1}: TEMU-Fulfilment détecté, PO-ID -> {po_id}")

            # Créer overlay avec image + PO-ID
            page_width = float(page.mediabox[2])
            page_height = float(page.mediabox[3])
            packet = io.BytesIO()
            can = canvas.Canvas(packet, pagesize=(page_width, page_height))

            # Ajouter image Caution
            img = ImageReader(image_path)
            img_width, img_height = img.getSize()
            scale = 0.07
            can.drawImage(image_path,
                          page_width - img_width * scale - 15,
                          page_height - img_height * scale - 30,
                          width=img_width * scale,
                          height=img_height * scale)

            # Ajouter PO-ID sous TEMU-Fulfilment
            if po_id:
                can.setFont("Helvetica-Bold", 14)
                can.drawString(10, page_height - 30, po_id)

            can.save()
            packet.seek(0)

            overlay_pdf = PyPDF2.PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)

            # Ajouter la page clonée pour éviter le flux fermé
            writer.add_page(clone_page(page))
            page_num += 2 if skip_next else 1
        else:
            writer.add_page(clone_page(page))
            page_num += 1

# Sauvegarder PDF intermédiaire
with open(output_pdf_path, "wb") as f_out:
    writer.write(f_out)
print(f"🎉 PDF généré avec image et PO-ID : {output_pdf_path}")

# ------------------ 2️⃣ Tri PDF selon le guide ------------------
# Extraire IDs du guide
guide_order = []
with open(guide_pdf_path, "rb") as guide_file:
    guide_reader = PyPDF2.PdfReader(guide_file)
    for g_page in guide_reader.pages:
        g_text = g_page.extract_text() or ""
        page_id = extract_id_from_page_text(g_text)
        if page_id:
            guide_order.append(page_id)

# Lire PDF final et associer pages à leurs IDs
with open(output_pdf_path, "rb") as final_file:
    final_reader = PyPDF2.PdfReader(final_file)
    pages_by_id = {}
    remaining_pages = []

    for page in final_reader.pages:
        text = page.extract_text() or ""
        page_id = extract_id_from_page_text(text)
        if page_id:
            pages_by_id.setdefault(page_id, []).append(page)
        else:
            remaining_pages.append(page)

# Créer PDF trié
writer_sorted = PyPDF2.PdfWriter()

# Ajouter pages dans l’ordre du guide
for page_id in guide_order:
    if page_id in pages_by_id:
        for p in pages_by_id[page_id]:
            writer_sorted.add_page(clone_page(p))
        del pages_by_id[page_id]

# Ajouter les pages restantes (IDs non dans guide)
for pages in pages_by_id.values():
    for p in pages:
        writer_sorted.add_page(clone_page(p))

# Ajouter les pages sans ID
for p in remaining_pages:
    writer_sorted.add_page(clone_page(p))

# Sauvegarder PDF trié
with open(sorted_output_path, "wb") as f_out:
    writer_sorted.write(f_out)

print(f"✅ PDF final trié par ID généré : {sorted_output_path}")


✔ Page 1: TEMU-Fulfilment détecté, PO-ID -> PO-210-15775625549432336
✔ Page 3: TEMU-Fulfilment détecté, PO-ID -> PO-210-15181099070071816
✔ Page 5: TEMU-Fulfilment détecté, PO-ID -> PO-210-15562256392313128
✔ Page 7: TEMU-Fulfilment détecté, PO-ID -> PO-210-04772829647992382
✔ Page 9: TEMU-Fulfilment détecté, PO-ID -> PO-210-04521368056953883
✔ Page 11: TEMU-Fulfilment détecté, PO-ID -> PO-210-04207613146231945
✔ Page 13: TEMU-Fulfilment détecté, PO-ID -> PO-210-04189501412472021
🎉 PDF généré avec image et PO-ID : data/output_final.pdf


ValueError: seek of closed file

In [ ]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
import re
from collections import defaultdict

input_pdf_path = "data/Temu _ Manage orders (1).pdf"
guide_pdf_path = "data/4.pdf"
image_path = "data/Caution (1).png"
output_pdf_path = "data/output_sorted.pdf"


def extract_po_id(text):
    match = re.search(r"PO-\d+-\d+", text)
    return match.group(0) if match else None


# Step 1: Process labels with image and PO-ID
temp_pdf_path = "data/temp_with_labels.pdf"

with open(input_pdf_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    writer = PyPDF2.PdfWriter()
    
    page_num = 0
    num_pages = len(reader.pages)
    
    while page_num < num_pages:
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        
        if "TEMU-Fulfilment" in text:
            po_id = None
            if page_num + 1 < num_pages:
                next_page = reader.pages[page_num + 1]
                next_text = next_page.extract_text() or ""
                
                if "PO-" in next_text:
                    po_id = extract_po_id(next_text)
                    skip_next = True
                else:
                    skip_next = False
            else:
                skip_next = False
            
            page_width = float(page.mediabox[2])
            page_height = float(page.mediabox[3])
            
            packet = io.BytesIO()
            can = canvas.Canvas(packet, pagesize=(page_width, page_height))
            
            img = ImageReader(image_path)
            img_width, img_height = img.getSize()
            
            scale = 0.07
            new_width = img_width * scale
            new_height = img_height * scale
            
            x_pos = page_width - new_width - 15
            y_pos = page_height - new_height - 30
            
            can.drawImage(image_path, x_pos, y_pos, width=new_width, height=new_height)
            
            if po_id:
                can.setFont("Helvetica-Bold", 14)
                can.drawString(10, page_height - 30, po_id)
            
            can.save()
            packet.seek(0)
            
            overlay_pdf = PyPDF2.PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)
            
            writer.add_page(page)
            
            if skip_next:
                page_num += 2
            else:
                page_num += 1
        else:
            writer.add_page(page)
            page_num += 1
    
    with open(temp_pdf_path, "wb") as out_file:
        writer.write(out_file)

print("✅ Step 1: Labels processed")


# Step 2: Extract order sequence from guide
guide_order_sequence = []

with open(guide_pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        
        po_id = extract_po_id(text)
        if po_id:
            guide_order_sequence.append(po_id)

print(f"✅ Step 2: Found {len(guide_order_sequence)} orders in guide")


# Step 3: Reorder - KEEP FILE OPEN DURING WRITE
with open(temp_pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    pages_by_id = defaultdict(list)
    remaining_pages = []
    
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        
        po_id = extract_po_id(text)
        
        if po_id:
            pages_by_id[po_id].append(page)
        else:
            remaining_pages.append(page)
    
    writer_sorted = PyPDF2.PdfWriter()
    
    # Add pages in guide order
    for po_id in guide_order_sequence:
        if po_id in pages_by_id:
            for p in pages_by_id[po_id]:
                writer_sorted.add_page(p)
    
    # Add remaining pages
    for p in remaining_pages:
        writer_sorted.add_page(p)
    
    # Write BEFORE closing file
    with open(output_pdf_path, "wb") as out_file:
        writer_sorted.write(out_file)

print(f"🎉 Final: {output_pdf_path}")

In [25]:
!pip install pikepdf

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 24.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
import re

# ------------------ PATHS ------------------
input_pdf_path = "data/Temu _ Manage orders (1).pdf"
image_path = "data/Caution (1).png"
output_pdf_path = "data/output_final.pdf"
guide_pdf_path = "data/4.pdf"
sorted_output_path = "data/output_sorted.pdf"

# ------------------ FUNCTIONS ------------------
def extract_id_from_text(text):
    """
    Extrait un ID sous forme de chiffres + tirets
    Exemple : 210-15775625549432336
    """
    match = re.search(r"\d+-[\d-]+", text)
    return match.group(0) if match else None


# ------------------ 1️⃣ Traitement PDF (ajout image + ID) ------------------
with open(input_pdf_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    writer = PyPDF2.PdfWriter()
    page_num = 0
    num_pages = len(reader.pages)

    while page_num < num_pages:
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        po_id = None
        skip_next = False

        if "TEMU-Fulfilment" in text:
            # Vérifier la page suivante pour l'ID
            if page_num + 1 < num_pages:
                next_page = reader.pages[page_num + 1]
                next_text = next_page.extract_text() or ""
                po_id = extract_id_from_text(next_text)
                if po_id:
                    skip_next = True
                    print(f"✔ Page {page_num+1}: TEMU-Fulfilment détecté, ID trouvé -> {po_id}")

            # Créer overlay avec image + ID
            page_width = float(page.mediabox[2])
            page_height = float(page.mediabox[3])
            packet = io.BytesIO()
            can = canvas.Canvas(packet, pagesize=(page_width, page_height))

            # Image Caution
            img = ImageReader(image_path)
            img_width, img_height = img.getSize()
            scale = 0.07
            can.drawImage(image_path,
                          page_width - img_width*scale - 15,
                          page_height - img_height*scale - 30,
                          width=img_width*scale,
                          height=img_height*scale)

            # ID sous TEMU-Fulfilment
            if po_id:
                can.setFont("Helvetica-Bold", 14)
                can.drawString(10, page_height - 30, po_id)

            can.save()
            packet.seek(0)
            overlay_pdf = PyPDF2.PdfReader(packet)
            overlay_page = overlay_pdf.pages[0]
            page.merge_page(overlay_page)

            writer.add_page(page)
            page_num += 2 if skip_next else 1
        else:
            writer.add_page(page)
            page_num += 1

# Sauvegarde PDF intermédiaire
with open(output_pdf_path, "wb") as f_out:
    writer.write(f_out)
print(f"🎉 PDF généré avec image et ID : {output_pdf_path}")

# ------------------ 2️⃣ Tri PDF selon le guide ------------------

# Extraire IDs du guide
guide_order = []
with open(guide_pdf_path, "rb") as guide_file:
    guide_reader = PyPDF2.PdfReader(guide_file)
    for g_page in guide_reader.pages:
        g_text = g_page.extract_text() or ""
        page_id = extract_id_from_text(g_text)
        if page_id:
            guide_order.append(page_id)

# Lire PDF final et associer pages à l'ID
# IMPORTANT : On garde le fichier ouvert pendant tout le processus de création du nouveau PDF
with open(output_pdf_path, "rb") as final_file:
    final_reader = PyPDF2.PdfReader(final_file)
    pages_by_id = {}
    remaining_pages = []

    # Classement des pages en mémoire
    for page in final_reader.pages:
        text = page.extract_text() or ""
        page_id = extract_id_from_text(text)
        if page_id:
            pages_by_id.setdefault(page_id, []).append(page)
        else:
            remaining_pages.append(page)

    # Créer PDF trié
    writer_sorted = PyPDF2.PdfWriter()

    # 1. Ajouter pages dans l’ordre du guide
    for page_id in guide_order:
        if page_id in pages_by_id:
            for p in pages_by_id[page_id]:
                writer_sorted.add_page(p)
            del pages_by_id[page_id] # On retire pour ne pas doublonner

    # 2. Ajouter pages restantes (IDs trouvés mais non présents dans le guide)
    for pages in pages_by_id.values():
        for p in pages:
            writer_sorted.add_page(p)

    # 3. Ajouter pages sans ID
    for p in remaining_pages:
        writer_sorted.add_page(p)

    # Sauvegarder PDF trié (ceci doit aussi être fait avant que final_file ne soit fermé par le bloc 'with')
    with open(sorted_output_path, "wb") as f_out:
        writer_sorted.write(f_out)

print(f"✅ PDF final trié par ID généré : {sorted_output_path}")

✔ Page 1: TEMU-Fulfilment détecté, ID trouvé -> 210-15775625549432336
✔ Page 3: TEMU-Fulfilment détecté, ID trouvé -> 210-15181099070071816
✔ Page 5: TEMU-Fulfilment détecté, ID trouvé -> 210-15562256392313128
✔ Page 7: TEMU-Fulfilment détecté, ID trouvé -> 210-04772829647992382
✔ Page 9: TEMU-Fulfilment détecté, ID trouvé -> 210-04521368056953883
✔ Page 11: TEMU-Fulfilment détecté, ID trouvé -> 210-04207613146231945
✔ Page 13: TEMU-Fulfilment détecté, ID trouvé -> 210-04189501412472021
🎉 PDF généré avec image et ID : data/output_final.pdf
✅ PDF final trié par ID généré : data/output_sorted.pdf


In [28]:
import PyPDF2
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io
import re
from PyPDF2 import PageObject

# ------------------ PATHS ------------------
input_pdf_path = "data/Temu _ Manage orders (1).pdf"
image_path = "data/Caution (1).png"
guide_pdf_path = "data/4.pdf"
sorted_output_path = "data/output_sorted.pdf"

# ------------------ FUNCTIONS ------------------
def extract_po_id(text):
    match = re.search(r"PO-\d+-\d+", text)
    return match.group(0) if match else None

def extract_id_from_text(text):
    match = re.search(r"\d+-[\d-]+", text)
    return match.group(0) if match else None

def clone_page(page):
    new_page = PageObject.create_blank_page(width=page.mediabox.width, height=page.mediabox.height)
    new_page.merge_page(page)
    return new_page

def create_overlay(width, height, po_id=None):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=(width, height))

    # Ajouter image
    img = ImageReader(image_path)
    img_width, img_height = img.getSize()
    scale = 0.07
    can.drawImage(image_path,
                  width - img_width * scale - 15,
                  height - img_height * scale - 30,
                  width=img_width * scale,
                  height=img_height * scale)

    # Ajouter texte PO-ID
    if po_id:
        can.setFont("Helvetica-Bold", 14)
        can.drawString(10, height - 30, po_id)

    can.save()
    packet.seek(0)
    overlay_pdf = PyPDF2.PdfReader(packet)
    return overlay_pdf.pages[0]

# ------------------ Extraire IDs du guide ------------------
guide_order = []
with open(guide_pdf_path, "rb") as guide_file:
    guide_reader = PyPDF2.PdfReader(guide_file)
    for g_page in guide_reader.pages:
        g_text = g_page.extract_text() or ""
        page_id = extract_id_from_text(g_text)
        if page_id:
            guide_order.append(page_id)

# ------------------ Traitement PDF principal + tri ------------------
with open(input_pdf_path, "rb") as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    pages_by_id = {}
    remaining_pages = []

    page_num = 0
    num_pages = len(reader.pages)

    while page_num < num_pages:
        page = reader.pages[page_num]
        text = page.extract_text() or ""
        po_id = None
        skip_next = False

        if "TEMU-Fulfilment" in text:
            # Vérifier page suivante pour PO-ID
            if page_num + 1 < num_pages:
                next_page = reader.pages[page_num + 1]
                next_text = next_page.extract_text() or ""
                po_id = extract_po_id(next_text)
                if po_id:
                    skip_next = True

            # Ajouter overlay
            page_width = float(page.mediabox[2])
            page_height = float(page.mediabox[3])
            overlay = create_overlay(page_width, page_height, po_id)
            page.merge_page(overlay)

            # Extraire ID pour le tri
            page_id = extract_id_from_text(po_id) if po_id else extract_id_from_text(text)
            if page_id:
                pages_by_id.setdefault(page_id, []).append(clone_page(page))
            else:
                remaining_pages.append(clone_page(page))

            page_num += 2 if skip_next else 1
        else:
            page_id = extract_id_from_text(text)
            if page_id:
                pages_by_id.setdefault(page_id, []).append(clone_page(page))
            else:
                remaining_pages.append(clone_page(page))
            page_num += 1

# ------------------ Créer PDF trié ------------------
writer_sorted = PyPDF2.PdfWriter()

# Ajouter pages selon l’ordre du guide
for page_id in guide_order:
    if page_id in pages_by_id:
        for p in pages_by_id[page_id]:
            writer_sorted.add_page(p)
        del pages_by_id[page_id]

# Ajouter pages restantes (IDs non dans guide)
for pages in pages_by_id.values():
    for p in pages:
        writer_sorted.add_page(p)

# Ajouter pages sans ID
for p in remaining_pages:
    writer_sorted.add_page(p)

# Sauvegarder PDF final
with open(sorted_output_path, "wb") as f_out:
    writer_sorted.write(f_out)

print(f"✅ PDF final traité et trié généré : {sorted_output_path}")


ValueError: seek of closed file